In [ ]:
import fnmatch
import os
import pathlib
import random
import time

import intel_extension_for_pytorch as ipex
import matplotlib.pyplot as plt
import numpy as np
import psutil
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from tqdm import tqdm

sns.set_theme(style="whitegrid", palette="husl")

In [ ]:
model_name = "resnet18_3Class_RN_epoch_10"
model_name = "bc_resnet18_simple_NOIPEX_6Epochs_gold"
model_read = torch.jit.load(f"models/{model_name}.pt")
model_read = torch.jit.freeze(model_read)
model_read.eval()

In [ ]:
model_read = ipex.optimize(model_read, dtype=torch.float32)
#model_read = model_read.to(memory_format=torch.channels_last)

In [ ]:
imagenet_stats = [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]

class ImageFolderWithPaths(datasets.ImageFolder):
    """custom ImageFolder to get the filepaths along with the image and label data."""

    def __getitem__(self, index):
        paths = ((self.imgs[index][0]),)
        return super().__getitem__(index) + paths


def infer(model, data_path: str):
    """give trained `model` & `data_path` where images whose
    labels have to be predicted are kept.

    `data_path`: path to data eg. ./test/<random_class>/*.png
    it's important to have a folder with a`random class` name as ImgFolder
    expects it.

    returns: (
        images: images loaded from disk for inferece,
        yhats: predicted labels
        paths: image file-path on disk        )
    """
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize(*imagenet_stats)]
    )
    data = ImageFolderWithPaths(data_path, transform=transform)
    dataloader = DataLoader(
        data,
        batch_size=4,
    )
    yhats = []
    images = []
    paths = []
    for (imgs, _, fpaths) in dataloader:
        yhat = model(imgs)
        yhat = yhat.max(1)[1]
        yhat = yhat.data.cpu().numpy()
        yhats.extend(yhat)
        paths.extend(fpaths)
        images.extend(imgs.data.cpu())
    return images, yhats, paths

def show_data(dataloader, imagenet_stats=imagenet_stats, num_data=4, figsize=(10, 6)):
    """show `num_data` of images and labels from dataloader."""
    batch = next(iter(dataloader))  # batch of with images, batch of labels
    imgs, labels = (
        batch[0][:num_data],
        batch[1][:num_data].tolist(),
    )  # get num_data of images, labels
    plt.style.use("dark_background")
    _, axes = plt.subplots(1, num_data, figsize=figsize)
    for n in range(num_data):
        axes[n].set_title(labels[n])
        imgs[n] = _denormalize(imgs[n], imagenet_stats)
        axes[n].imshow(torch.clamp(imgs[n], 0, 1).permute(1, 2, 0))

def _denormalize(images, imagenet_stats):
    """de normalize dataset using imagenet std and mean to show images"""
    mean = torch.tensor(imagenet_stats[0]).reshape(1, 3, 1, 1)
    std = torch.tensor(imagenet_stats[1]).reshape(1, 3, 1, 1)
    return images * std + mean

In [ ]:
!mkdir -p data/ThreeClassManualRemove0s/test
!mkdir  -p data/ThreeClassManualRemove0s/test/unknown
#!cp data/DinosaurNationalMonument/20220514/224/*.jpg data/ThreeClassManualRemove0s/test/unknown/

In [ ]:
imagenet_stats = [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]
images, yhats, img_paths = infer(
    model_read, data_path="./data/ThreeClassManualRemove0s/test/"
)
infer_dataloader = DataLoader([*zip(images, yhats)], batch_size=100, shuffle=False)
print("infered images with labels")
show_data(infer_dataloader, imagenet_stats, 10, figsize=(20, 8))


In [ ]:
import glob
from PIL import Image
from os.path import exists
import os
path = "data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png"
img = Image.open(path)


# Get sample near DNM Excavation

In [ ]:
left = 4*224+112
low = 4*224+112
bbox = (left, low, left + 224, low + 224)
working_slice = img.crop(bbox)
working_slice

# Get sample at DNM near top of image

In [ ]:
left = 4*224+112
low = 0*224+112
bbox = (left, low, left + 224, low + 224)
working_slice = img.crop(bbox)
working_slice

# Simple eval function

In [ ]:
model_read = model_read.float()

In [ ]:
from PIL import Image
import torchvision.transforms.functional as tvision

left = 4*224
low = 4*224
bbox = (left, low, left + 224, low + 224)
working_slice = img.crop(bbox)
working_slice

def eval_simple(working_slice):
    x = tvision.to_tensor(working_slice)
    x.unsqueeze_(0)
    x = x.to(memory_format=torch.channels_last)
    output = model_read(x)
    return output#.detach().numpy().argmax()

def eval_batch(working_slices):
    x = torch.from_numpy(working_slices)
    #x.unsqueeze_(0)
    x = x.to(memory_format=torch.channels_last)
    outputs = model_read(x)
    return outputs#.detach().numpy().argmax()


eval_simple(working_slice)

# Score large map of DNM

<img src="data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png" width="500"/>


In [ ]:
import time
from PIL import Image

step = 28
# counts = {0:0, 1:0, 2:0}
# scale = {0:-10, 1:0, 2:10}


def img_crops(img):
    working_slices = []
    for x in tqdm(range(0, img.size[0]-224, step)):
        for y in range(0, img.size[1]-224, step):  
            bbox = (x, y, x + 224, y + 224)
            working_slices.append(np.asarray(img.crop(bbox), dtype="float32"))
    return working_slices


working_slices = img_crops(img)

class SlicesDataset(torch.utils.data.Dataset):
    def __init__(self, img_list):
        super().__init__()
        self.img_list = img_list
        self.transform = transforms.Compose(
            [transforms.ToTensor(), 
             transforms.Normalize(*imagenet_stats)])

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img = self.img_list[idx]
        return self.transform(img)
  


In [ ]:
import os
import torch

torch.set_num_threads(8)
torch.set_num_interop_threads(2)
print(torch.get_num_interop_threads())
print(torch.get_num_threads())
os.environ["KMP_AFFINITY"] = "granularity=fine,compact,1,0"
st_time = time.time()
yhats = []
for patch in DataLoader(SlicesDataset(working_slices), batch_size=8):
    yhat = model_read(patch)
    yhat = torch.log_softmax(yhat, dim=1)  # softmax of logit output
    yhat = yhat.max(1)[1]
    yhats.append(yhat)
print(f"total time: {time.time() - st_time}")

In [ ]:
model_read(patch)

In [ ]:

# import time

# from PIL import Image
# countBuf = np.ones( (img.size[0], img.size[1]) )
# sumBuf =   np.zeros( (img.size[0], img.size[1]) )

# start = time.time()
# step = 28
# counts = {0:0, 1:0, 2:0}
# #scale = {0:-10, 1:0, 2:10}
# for x in tqdm(range(0, img.size[0]-224, step)):
#     for y in range(0, img.size[1]-224, step):  
#         bbox = (x, y, x + 224, y + 224)
#         working_slice = img.crop(bbox)
#         countBuf[bbox[0]:bbox[2], bbox[1]:bbox[3]] += 1
#         sumBuf[bbox[0]:bbox[2], bbox[1]:bbox[3]] += eval_simple(working_slice)
#         counts[eval_simple(working_slice)] += 1
#         #print(x, y, eval_simple(working_slice))
# print(f"step size = {step}, Elapsed: {(time.time() - start):6.1f} sec")
# counts


In [ ]:
#%load_ext line_profiler

In [ ]:
import os
torch.set_num_threads(8)
torch.set_num_interop_threads(1)
print(torch.get_num_interop_threads())
print(torch.get_num_threads())
os.environ["KMP_AFFINITY"] = "granularity=fine,compact,1,0"
#%lprun -f xy xy()

# Compute the mean of scores for sliding tiles

In [ ]:
"""
#set transparency to 159: 0 is transparent, 255 is opaque
meanBuf = sumBuf/countBuf
mat = np.uint8(meanBuf.T*159/meanBuf.max())
output = Image.fromarray(mat)
output.save('data/bobTile.png')
np.save('data/meanBuf.npy', mat)
"""

# Create RGBA image with green transparency mask

In [ ]:
"""
from PIL import Image
path = "data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png"
img = Image.open(path)
green = Image.new('RGBA',meanBuf.shape,(0,255,0,255))
alpha = np.uint8(meanBuf.T*159/meanBuf.max())
#green.putalpha(alpha)
mask = Image.fromarray(alpha)
img.paste(green, (0, 0), mask)
img.save('data/bobNewMap.png')
img
"""

In [ ]:
#np.histogram(alpha, bins=8)

# NumPy Select: Color threshold differently

In [ ]:
"""
import time

from PIL import Image
path = "data/DinosaurNationalMonument/Dinosaur National Monument Panorama.png"
img = Image.open(path)
condition = [ (alpha  < 100) ,
              ( alpha >= 100) & (alpha < 140)]
choice = [ 0, 
           64 ]
default = [(128)]
newAlpha = np.select(condition, choice, default= default )

mask = Image.fromarray(newAlpha.astype('uint8'))
img.paste(green, (0, 0), mask)
img.save('data/bobNewMapSelect.png')
img
"""

# Heatmap Approach

Color Legend:
- Bright Red: Bones more likely
- Bright Blue: Bone not likely

In [ ]:
"""
from PIL import ImageDraw
import numpy as np
import cv2

alpha = np.load('data/meanBuf.npy')
caption = ImageDraw.Draw(img)
imgcv = cv2.imread(path)
heatmap_img = np.copy(imgcv)
alpha1D = alpha/alpha.max()*255
heatmap_img[:,:,0] = alpha1D
heatmap_img[:,:,1] = 0
heatmap_img[:,:,2] = 255 - alpha1D
super_imposed_img = cv2.addWeighted(heatmap_img, 0.7, imgcv, 0.3, 0)
imgHeat = Image.fromarray(super_imposed_img.astype('uint8'))
imgHeat.save('data/bobHeat.png')

imgHeat.show()
print("Color Legend:\n- Red: Higher Probability\n- Blue: -Lower Probability")
"""